In [1]:
# allocate fraction of GPU memory 
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf 
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
import pandas as pd
import gc
from keras import optimizers, losses, activations, models
from keras.layers import Dense, Input, Flatten, Dropout, InputLayer
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras import backend as K

In [3]:
import numpy as np
import timeit
import pickle

In [4]:
with open('feature_arr', 'rb') as fo:
        feature = pickle.load(fo)
with open('y_train', 'rb') as fo:
        feature_label = pickle.load(fo)
with open('feature_arr_test', 'rb') as fo:
        feature_test = pickle.load(fo)
with open('y_test', 'rb') as fo:
        feature_label_test = pickle.load(fo)

# first layer model

In [5]:
K.clear_session()
dense_model = models.Sequential()
dense_model.add(Flatten(input_shape=(1,4096)))
dense_model.add(Dense(4096, activation='relu'))
dense_model.add(Dense(100, activation='softmax'))
dense_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_2 (Dense)              (None, 100)               409700    
Total params: 17,191,012
Trainable params: 17,191,012
Non-trainable params: 0
_________________________________________________________________


In [6]:
x_train, x_valid, y_train, y_valid = train_test_split(feature, feature_label, test_size=0.1, random_state=42)

In [7]:
x_valid.shape

(5000, 4096)

In [8]:
x_train=np.expand_dims(x_train, axis=1)
x_valid=np.expand_dims(x_valid, axis=1)

In [9]:
dense_model.compile(loss=losses.categorical_crossentropy,optimizer=optimizers.Adadelta(),metrics=['accuracy'])

In [11]:
start = timeit.default_timer() 
single_layer_0_history=dense_model.fit(x_train, y_train,batch_size=1024,epochs=50,verbose=1,validation_data=(x_valid, y_valid), shuffle=True)
dense_model.save('single_layer_0')
dense_model.save('single_layer_0.h5')
end = timeit.default_timer()
print("Time Taken to run the model:",end - start, "seconds")

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 22s 480us/step - loss: 0.0756 - acc: 0.9852 - val_loss: 0.8693 - val_acc: 0.7490
Epoch 2/50
45000/45000 [==============================] - 22s 479us/step - loss: 0.0187 - acc: 0.9992 - val_loss: 0.8620 - val_acc: 0.7512
Epoch 3/50
45000/45000 [==============================] - 22s 478us/step - loss: 0.0123 - acc: 0.9994 - val_loss: 0.8636 - val_acc: 0.7538
Epoch 4/50
45000/45000 [==============================] - 22s 483us/step - loss: 0.0098 - acc: 0.9994 - val_loss: 0.8608 - val_acc: 0.7546
Epoch 5/50
45000/45000 [==============================] - 22s 479us/step - loss: 0.0091 - acc: 0.9993 - val_loss: 0.8634 - val_acc: 0.7560
Epoch 6/50
45000/45000 [==============================] - 21s 476us/step - loss: 0.0071 - acc: 0.9995 - val_loss: 0.8589 - val_acc: 0.7580
Epoch 7/50
45000/45000 [==============================] - 22s 480us/step - loss: 0.0072 - acc: 0.9994 - val_loss: 0.8

In [11]:
with open('single_layer_0_history', 'wb') as fo:
        pickle.dump(single_layer_0_history,fo)

In [12]:
model_0 = Model(inputs=dense_model.input, outputs=dense_model.get_layer('dense_1').output)

In [13]:
def get_feture(x,model):
    start = timeit.default_timer() 
    #p=model.predict(x)
    p=model.predict_on_batch(x)
    end = timeit.default_timer()
    print("Time Taken to get prediction:",end - start, "seconds")
    return p

In [14]:
inputto_2=get_feture(x_train[0:100],model_0)
inputto_2.shape

Time Taken to get prediction: 0.3810043670000596 seconds


(100, 4096)

In [15]:
 def get_prediction(input_arr,output_arr,model):
        for i in range(1,int(input_arr.shape[0]/100)):
            var=input_arr[(100*i):(i+1)*100]
            out=get_feture(var,model)
            output_arr=np.vstack((output_arr,out))
            print("Done{} feture{} shape now{}".format(i,out.shape,output_arr.shape))
        return output_arr

In [16]:
inputto_2=get_prediction(x_train,inputto_2,model_0)

Time Taken to get prediction: 0.013453758999958154 seconds
Done1 feture(100, 4096) shape now(200, 4096)
Time Taken to get prediction: 0.012003444000129093 seconds
Done2 feture(100, 4096) shape now(300, 4096)
Time Taken to get prediction: 0.01198933999990004 seconds
Done3 feture(100, 4096) shape now(400, 4096)
Time Taken to get prediction: 0.011915994999981194 seconds
Done4 feture(100, 4096) shape now(500, 4096)
Time Taken to get prediction: 0.01209900399999242 seconds
Done5 feture(100, 4096) shape now(600, 4096)
Time Taken to get prediction: 0.011981229999946663 seconds
Done6 feture(100, 4096) shape now(700, 4096)
Time Taken to get prediction: 0.011973119000003862 seconds
Done7 feture(100, 4096) shape now(800, 4096)
Time Taken to get prediction: 0.011895191000121486 seconds
Done8 feture(100, 4096) shape now(900, 4096)
Time Taken to get prediction: 0.012071851999962746 seconds
Done9 feture(100, 4096) shape now(1000, 4096)
Time Taken to get prediction: 0.013521460999982082 seconds
Done10

Done80 feture(100, 4096) shape now(8100, 4096)
Time Taken to get prediction: 0.012128975999985414 seconds
Done81 feture(100, 4096) shape now(8200, 4096)
Time Taken to get prediction: 0.012090893000049618 seconds
Done82 feture(100, 4096) shape now(8300, 4096)
Time Taken to get prediction: 0.01233702100012124 seconds
Done83 feture(100, 4096) shape now(8400, 4096)
Time Taken to get prediction: 0.012075377999963166 seconds
Done84 feture(100, 4096) shape now(8500, 4096)
Time Taken to get prediction: 0.012047874000018055 seconds
Done85 feture(100, 4096) shape now(8600, 4096)
Time Taken to get prediction: 0.012180811000007452 seconds
Done86 feture(100, 4096) shape now(8700, 4096)
Time Taken to get prediction: 0.012467488000083904 seconds
Done87 feture(100, 4096) shape now(8800, 4096)
Time Taken to get prediction: 0.01202848000002632 seconds
Done88 feture(100, 4096) shape now(8900, 4096)
Time Taken to get prediction: 0.01256657399994765 seconds
Done89 feture(100, 4096) shape now(9000, 4096)
Ti

Done158 feture(100, 4096) shape now(15900, 4096)
Time Taken to get prediction: 0.013761946999920838 seconds
Done159 feture(100, 4096) shape now(16000, 4096)
Time Taken to get prediction: 0.013467863999949259 seconds
Done160 feture(100, 4096) shape now(16100, 4096)
Time Taken to get prediction: 0.012493581999933667 seconds
Done161 feture(100, 4096) shape now(16200, 4096)
Time Taken to get prediction: 0.016067711000005147 seconds
Done162 feture(100, 4096) shape now(16300, 4096)
Time Taken to get prediction: 0.012270376000060423 seconds
Done163 feture(100, 4096) shape now(16400, 4096)
Time Taken to get prediction: 0.022866172000021834 seconds
Done164 feture(100, 4096) shape now(16500, 4096)
Time Taken to get prediction: 0.012971732000096381 seconds
Done165 feture(100, 4096) shape now(16600, 4096)
Time Taken to get prediction: 0.012831036999841672 seconds
Done166 feture(100, 4096) shape now(16700, 4096)
Time Taken to get prediction: 0.012440337000043655 seconds
Done167 feture(100, 4096) sh

Done234 feture(100, 4096) shape now(23500, 4096)
Time Taken to get prediction: 0.01361949000011009 seconds
Done235 feture(100, 4096) shape now(23600, 4096)
Time Taken to get prediction: 0.015522565000082977 seconds
Done236 feture(100, 4096) shape now(23700, 4096)
Time Taken to get prediction: 0.012903322999818556 seconds
Done237 feture(100, 4096) shape now(23800, 4096)
Time Taken to get prediction: 0.014012656999966566 seconds
Done238 feture(100, 4096) shape now(23900, 4096)
Time Taken to get prediction: 0.014294398999936675 seconds
Done239 feture(100, 4096) shape now(24000, 4096)
Time Taken to get prediction: 0.013963291999971261 seconds
Done240 feture(100, 4096) shape now(24100, 4096)
Time Taken to get prediction: 0.014379379000047265 seconds
Done241 feture(100, 4096) shape now(24200, 4096)
Time Taken to get prediction: 0.01395518100002846 seconds
Done242 feture(100, 4096) shape now(24300, 4096)
Time Taken to get prediction: 0.017817750000176602 seconds
Done243 feture(100, 4096) shap

Done311 feture(100, 4096) shape now(31200, 4096)
Time Taken to get prediction: 0.01376512100000582 seconds
Done312 feture(100, 4096) shape now(31300, 4096)
Time Taken to get prediction: 0.015026432000013301 seconds
Done313 feture(100, 4096) shape now(31400, 4096)
Time Taken to get prediction: 0.015062751999948887 seconds
Done314 feture(100, 4096) shape now(31500, 4096)
Time Taken to get prediction: 0.014329660000157673 seconds
Done315 feture(100, 4096) shape now(31600, 4096)
Time Taken to get prediction: 0.014147357999945598 seconds
Done316 feture(100, 4096) shape now(31700, 4096)
Time Taken to get prediction: 0.014504206000083286 seconds
Done317 feture(100, 4096) shape now(31800, 4096)
Time Taken to get prediction: 0.01389946700010114 seconds
Done318 feture(100, 4096) shape now(31900, 4096)
Time Taken to get prediction: 0.01497741799994401 seconds
Done319 feture(100, 4096) shape now(32000, 4096)
Time Taken to get prediction: 0.019662643999936336 seconds
Done320 feture(100, 4096) shape

Done388 feture(100, 4096) shape now(38900, 4096)
Time Taken to get prediction: 0.012061272999972061 seconds
Done389 feture(100, 4096) shape now(39000, 4096)
Time Taken to get prediction: 0.01224322400003075 seconds
Done390 feture(100, 4096) shape now(39100, 4096)
Time Taken to get prediction: 0.012368051000066771 seconds
Done391 feture(100, 4096) shape now(39200, 4096)
Time Taken to get prediction: 0.012334552000083931 seconds
Done392 feture(100, 4096) shape now(39300, 4096)
Time Taken to get prediction: 0.011994275999995807 seconds
Done393 feture(100, 4096) shape now(39400, 4096)
Time Taken to get prediction: 0.012454793999950198 seconds
Done394 feture(100, 4096) shape now(39500, 4096)
Time Taken to get prediction: 0.012153658999977779 seconds
Done395 feture(100, 4096) shape now(39600, 4096)
Time Taken to get prediction: 0.012856778000013946 seconds
Done396 feture(100, 4096) shape now(39700, 4096)
Time Taken to get prediction: 0.012007323000034376 seconds
Done397 feture(100, 4096) sha

In [17]:
inputto_2_valid=get_feture(x_valid[0:100],model_0)
inputto_2_valid=get_prediction(x_valid,inputto_2_valid,model_0)

Time Taken to get prediction: 0.3255789269999241 seconds
Time Taken to get prediction: 0.011877559999902587 seconds
Done1 feture(100, 4096) shape now(200, 4096)
Time Taken to get prediction: 0.011805626000068514 seconds
Done2 feture(100, 4096) shape now(300, 4096)
Time Taken to get prediction: 0.012077493999868238 seconds
Done3 feture(100, 4096) shape now(400, 4096)
Time Taken to get prediction: 0.012036238000064259 seconds
Done4 feture(100, 4096) shape now(500, 4096)
Time Taken to get prediction: 0.011895190999894112 seconds
Done5 feture(100, 4096) shape now(600, 4096)
Time Taken to get prediction: 0.012264029000107257 seconds
Done6 feture(100, 4096) shape now(700, 4096)
Time Taken to get prediction: 0.012261912999974811 seconds
Done7 feture(100, 4096) shape now(800, 4096)
Time Taken to get prediction: 0.012020017000168082 seconds
Done8 feture(100, 4096) shape now(900, 4096)
Time Taken to get prediction: 0.012078198999915912 seconds
Done9 feture(100, 4096) shape now(1000, 4096)
Time T

# second layer model

In [18]:
K.clear_session()
dense_model = models.Sequential()
dense_model.add(Flatten(input_shape=(1,4096)))
dense_model.add(Dense(4096, activation='relu'))
dense_model.add(Dense(100, activation='softmax'))
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_2 (Dense)              (None, 100)               409700    
Total params: 17,191,012
Trainable params: 17,191,012
Non-trainable params: 0
_________________________________________________________________


In [19]:
dense_model.compile(loss=losses.categorical_crossentropy,optimizer=optimizers.Adadelta(),metrics=['accuracy'])

In [20]:
inputto_2=np.expand_dims(inputto_2, axis=1)
inputto_2_valid=np.expand_dims(inputto_2_valid, axis=1)

In [22]:
start = timeit.default_timer() 
single_layer_1_history=dense_model.fit(inputto_2, y_train,batch_size=512,epochs=50,verbose=1,validation_data=(inputto_2_valid, y_valid), shuffle=True)
dense_model.save('single_layer_1')
dense_model.save('single_layer_1.h5')
end = timeit.default_timer()
print("Time Taken to run the model:",end - start, "seconds")

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 35s 785us/step - loss: 1.2796 - acc: 0.6956 - val_loss: 1.4406 - val_acc: 0.6444
Epoch 2/50
45000/45000 [==============================] - 36s 792us/step - loss: 0.1073 - acc: 0.9771 - val_loss: 1.0233 - val_acc: 0.7316
Epoch 3/50
45000/45000 [==============================] - 36s 791us/step - loss: 0.1232 - acc: 0.9796 - val_loss: 1.0023 - val_acc: 0.7364
Epoch 4/50
45000/45000 [==============================] - 35s 782us/step - loss: 0.0111 - acc: 0.9994 - val_loss: 0.9816 - val_acc: 0.7502
Epoch 5/50
45000/45000 [==============================] - 35s 783us/step - loss: 0.0039 - acc: 0.9997 - val_loss: 0.9994 - val_acc: 0.7520
Epoch 6/50
45000/45000 [==============================] - 35s 778us/step - loss: 0.0229 - acc: 0.9977 - val_loss: 1.0970 - val_acc: 0.7490
Epoch 7/50
45000/45000 [==============================] - 35s 784us/step - loss: 0.1682 - acc: 0.9894 - val_loss: 1.1

In [31]:
with open('single_layer_1_history', 'wb') as fo:
        pickle.dump(single_layer_1_history,fo)

## getting output from dense layer

In [23]:
model_0 = Model(inputs=dense_model.input, outputs=dense_model.get_layer('dense_1').output)

In [24]:
gc.collect()

346

In [25]:
inputto_3=get_feture(inputto_2[0:100],model_0)
inputto_3=get_prediction(inputto_2,inputto_3,model_0)

Time Taken to get prediction: 0.3823771059996943 seconds
Time Taken to get prediction: 0.011860987000090972 seconds
Done1 feture(100, 4096) shape now(200, 4096)
Time Taken to get prediction: 0.011864159999731783 seconds
Done2 feture(100, 4096) shape now(300, 4096)
Time Taken to get prediction: 0.011924106000151369 seconds
Done3 feture(100, 4096) shape now(400, 4096)
Time Taken to get prediction: 0.012035884999932023 seconds
Done4 feture(100, 4096) shape now(500, 4096)
Time Taken to get prediction: 0.011879322999902797 seconds
Done5 feture(100, 4096) shape now(600, 4096)
Time Taken to get prediction: 0.011827136000192695 seconds
Done6 feture(100, 4096) shape now(700, 4096)
Time Taken to get prediction: 0.011764370000037161 seconds
Done7 feture(100, 4096) shape now(800, 4096)
Time Taken to get prediction: 0.011851113999910012 seconds
Done8 feture(100, 4096) shape now(900, 4096)
Time Taken to get prediction: 0.012380040000152803 seconds
Done9 feture(100, 4096) shape now(1000, 4096)
Time T

Done80 feture(100, 4096) shape now(8100, 4096)
Time Taken to get prediction: 0.012954453000020294 seconds
Done81 feture(100, 4096) shape now(8200, 4096)
Time Taken to get prediction: 0.013184360000195738 seconds
Done82 feture(100, 4096) shape now(8300, 4096)
Time Taken to get prediction: 0.01306870100006563 seconds
Done83 feture(100, 4096) shape now(8400, 4096)
Time Taken to get prediction: 0.012845141999605403 seconds
Done84 feture(100, 4096) shape now(8500, 4096)
Time Taken to get prediction: 0.012441748000128428 seconds
Done85 feture(100, 4096) shape now(8600, 4096)
Time Taken to get prediction: 0.012294001000100252 seconds
Done86 feture(100, 4096) shape now(8700, 4096)
Time Taken to get prediction: 0.012631455000246206 seconds
Done87 feture(100, 4096) shape now(8800, 4096)
Time Taken to get prediction: 0.012140611999711837 seconds
Done88 feture(100, 4096) shape now(8900, 4096)
Time Taken to get prediction: 0.01245267900003455 seconds
Done89 feture(100, 4096) shape now(9000, 4096)
T

Done157 feture(100, 4096) shape now(15800, 4096)
Time Taken to get prediction: 0.012249923000126728 seconds
Done158 feture(100, 4096) shape now(15900, 4096)
Time Taken to get prediction: 0.012164237999968464 seconds
Done159 feture(100, 4096) shape now(16000, 4096)
Time Taken to get prediction: 0.01534942999978739 seconds
Done160 feture(100, 4096) shape now(16100, 4096)
Time Taken to get prediction: 0.012538364999727492 seconds
Done161 feture(100, 4096) shape now(16200, 4096)
Time Taken to get prediction: 0.02100893700026063 seconds
Done162 feture(100, 4096) shape now(16300, 4096)
Time Taken to get prediction: 0.012082431000180804 seconds
Done163 feture(100, 4096) shape now(16400, 4096)
Time Taken to get prediction: 0.01201613900002485 seconds
Done164 feture(100, 4096) shape now(16500, 4096)
Time Taken to get prediction: 0.01727824599993255 seconds
Done165 feture(100, 4096) shape now(16600, 4096)
Time Taken to get prediction: 0.013669912999830558 seconds
Done166 feture(100, 4096) shape 

Done234 feture(100, 4096) shape now(23500, 4096)
Time Taken to get prediction: 0.012904735000120127 seconds
Done235 feture(100, 4096) shape now(23600, 4096)
Time Taken to get prediction: 0.012996767999993608 seconds
Done236 feture(100, 4096) shape now(23700, 4096)
Time Taken to get prediction: 0.012136027999986254 seconds
Done237 feture(100, 4096) shape now(23800, 4096)
Time Taken to get prediction: 0.012764392999997654 seconds
Done238 feture(100, 4096) shape now(23900, 4096)
Time Taken to get prediction: 0.013623015000121086 seconds
Done239 feture(100, 4096) shape now(24000, 4096)
Time Taken to get prediction: 0.014582487000097899 seconds
Done240 feture(100, 4096) shape now(24100, 4096)
Time Taken to get prediction: 0.012815170000067155 seconds
Done241 feture(100, 4096) shape now(24200, 4096)
Time Taken to get prediction: 0.012759455999912461 seconds
Done242 feture(100, 4096) shape now(24300, 4096)
Time Taken to get prediction: 0.013826122999944346 seconds
Done243 feture(100, 4096) sh

Done310 feture(100, 4096) shape now(31100, 4096)
Time Taken to get prediction: 0.01246678300003623 seconds
Done311 feture(100, 4096) shape now(31200, 4096)
Time Taken to get prediction: 0.012016138000035426 seconds
Done312 feture(100, 4096) shape now(31300, 4096)
Time Taken to get prediction: 0.011985812999682821 seconds
Done313 feture(100, 4096) shape now(31400, 4096)
Time Taken to get prediction: 0.012402253999880486 seconds
Done314 feture(100, 4096) shape now(31500, 4096)
Time Taken to get prediction: 0.012609945999884076 seconds
Done315 feture(100, 4096) shape now(31600, 4096)
Time Taken to get prediction: 0.01292095500002688 seconds
Done316 feture(100, 4096) shape now(31700, 4096)
Time Taken to get prediction: 0.013166376000299351 seconds
Done317 feture(100, 4096) shape now(31800, 4096)
Time Taken to get prediction: 0.012258739000117203 seconds
Done318 feture(100, 4096) shape now(31900, 4096)
Time Taken to get prediction: 0.012410012000145798 seconds
Done319 feture(100, 4096) shap

Done386 feture(100, 4096) shape now(38700, 4096)
Time Taken to get prediction: 0.012258385999757593 seconds
Done387 feture(100, 4096) shape now(38800, 4096)
Time Taken to get prediction: 0.012991125000098691 seconds
Done388 feture(100, 4096) shape now(38900, 4096)
Time Taken to get prediction: 0.012227708999944298 seconds
Done389 feture(100, 4096) shape now(39000, 4096)
Time Taken to get prediction: 0.012682938000125432 seconds
Done390 feture(100, 4096) shape now(39100, 4096)
Time Taken to get prediction: 0.01581770500024504 seconds
Done391 feture(100, 4096) shape now(39200, 4096)
Time Taken to get prediction: 0.015142796000418457 seconds
Done392 feture(100, 4096) shape now(39300, 4096)
Time Taken to get prediction: 0.015766222999900492 seconds
Done393 feture(100, 4096) shape now(39400, 4096)
Time Taken to get prediction: 0.018327282999962335 seconds
Done394 feture(100, 4096) shape now(39500, 4096)
Time Taken to get prediction: 0.01534343599996646 seconds
Done395 feture(100, 4096) shap

In [26]:
inputto_3_valid=get_feture(inputto_2_valid[0:100],model_0)
inputto_3_valid=get_prediction(inputto_2_valid,inputto_3_valid,model_0)

Time Taken to get prediction: 0.34265512399997533 seconds
Time Taken to get prediction: 0.011882848999903217 seconds
Done1 feture(100, 4096) shape now(200, 4096)
Time Taken to get prediction: 0.01190188999999009 seconds
Done2 feture(100, 4096) shape now(300, 4096)
Time Taken to get prediction: 0.011973472000136098 seconds
Done3 feture(100, 4096) shape now(400, 4096)
Time Taken to get prediction: 0.012023190000036266 seconds
Done4 feture(100, 4096) shape now(500, 4096)
Time Taken to get prediction: 0.011886375000358385 seconds
Done5 feture(100, 4096) shape now(600, 4096)
Time Taken to get prediction: 0.011966419000145834 seconds
Done6 feture(100, 4096) shape now(700, 4096)
Time Taken to get prediction: 0.01189942200016958 seconds
Done7 feture(100, 4096) shape now(800, 4096)
Time Taken to get prediction: 0.01196395199986 seconds
Done8 feture(100, 4096) shape now(900, 4096)
Time Taken to get prediction: 0.012044699999933073 seconds
Done9 feture(100, 4096) shape now(1000, 4096)
Time Taken 

In [27]:
inputto_3=np.expand_dims(inputto_3, axis=1)
inputto_3_valid=np.expand_dims(inputto_3_valid, axis=1)

In [37]:
y_valid.shape

(5000, 100)

# third layer model

In [29]:
K.clear_session()
dense_model = models.Sequential()
dense_model.add(Flatten(input_shape=(1,4096)))
dense_model.add(Dense(4096, activation='relu'))
dense_model.add(Dense(100, activation='softmax'))
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_2 (Dense)              (None, 100)               409700    
Total params: 17,191,012
Trainable params: 17,191,012
Non-trainable params: 0
_________________________________________________________________


In [30]:
dense_model.compile(loss=losses.categorical_crossentropy,optimizer=optimizers.Adadelta(),metrics=['accuracy'])

In [33]:
start = timeit.default_timer() 
single_layer_2_history=dense_model.fit(inputto_3, y_train,batch_size=256,epochs=50,verbose=1,validation_data=(inputto_3_valid, y_valid), shuffle=True)
dense_model.save('single_layer_2')
dense_model.save('single_layer_2.h5')
end = timeit.default_timer()
print("Time Taken to run the model:",end - start, "seconds")

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 64s 1ms/step - loss: 14.4981 - acc: 0.1005 - val_loss: 14.6215 - val_acc: 0.0872
Epoch 2/50
45000/45000 [==============================] - 62s 1ms/step - loss: 14.4981 - acc: 0.1005 - val_loss: 14.6202 - val_acc: 0.0874
Epoch 3/50
45000/45000 [==============================] - 62s 1ms/step - loss: 14.4981 - acc: 0.1005 - val_loss: 14.6204 - val_acc: 0.0876
Epoch 4/50
45000/45000 [==============================] - 62s 1ms/step - loss: 14.4981 - acc: 0.1005 - val_loss: 14.6203 - val_acc: 0.0872
Epoch 5/50
45000/45000 [==============================] - 62s 1ms/step - loss: 14.4981 - acc: 0.1005 - val_loss: 14.6214 - val_acc: 0.0872
Epoch 6/50
45000/45000 [==============================] - 62s 1ms/step - loss: 14.4981 - acc: 0.1005 - val_loss: 14.6212 - val_acc: 0.0874
Epoch 7/50
45000/45000 [==============================] - 62s 1ms/step - loss: 14.4981 - acc: 0.1005 - val_loss: 14.6

KeyboardInterrupt: 

In [50]:
with open('single_layer_2_history', 'wb') as fo:
        pickle.dump(single_layer_2_history,fo)

# output from third layer

In [51]:
model_0 = Model(inputs=dense_model.input, outputs=dense_model.get_layer('dense_1').output)

In [52]:
inputto_4=get_feture(inputto_3[0:100],model_0)
inputto_4=get_prediction(inputto_3,inputto_4,model_0)

Time Taken to get prediction: 0.4032610680005746 seconds
Time Taken to get prediction: 0.003689426999699208 seconds
Done1 feture(100, 1024) shape now(200, 1024)
Time Taken to get prediction: 0.003409801000088919 seconds
Done2 feture(100, 1024) shape now(300, 1024)
Time Taken to get prediction: 0.0032620550009596627 seconds
Done3 feture(100, 1024) shape now(400, 1024)
Time Taken to get prediction: 0.0033858240003610263 seconds
Done4 feture(100, 1024) shape now(500, 1024)
Time Taken to get prediction: 0.0035194659994886024 seconds
Done5 feture(100, 1024) shape now(600, 1024)
Time Taken to get prediction: 0.0034253169997100485 seconds
Done6 feture(100, 1024) shape now(700, 1024)
Time Taken to get prediction: 0.0032969639996736078 seconds
Done7 feture(100, 1024) shape now(800, 1024)
Time Taken to get prediction: 0.0031538019993604394 seconds
Done8 feture(100, 1024) shape now(900, 1024)
Time Taken to get prediction: 0.0030628269996668678 seconds
Done9 feture(100, 1024) shape now(1000, 1024)

Done81 feture(100, 1024) shape now(8200, 1024)
Time Taken to get prediction: 0.003895001998898806 seconds
Done82 feture(100, 1024) shape now(8300, 1024)
Time Taken to get prediction: 0.0037380880003183847 seconds
Done83 feture(100, 1024) shape now(8400, 1024)
Time Taken to get prediction: 0.0031668490009906236 seconds
Done84 feture(100, 1024) shape now(8500, 1024)
Time Taken to get prediction: 0.0031396969989145873 seconds
Done85 feture(100, 1024) shape now(8600, 1024)
Time Taken to get prediction: 0.003177779999532504 seconds
Done86 feture(100, 1024) shape now(8700, 1024)
Time Taken to get prediction: 0.0031421660005435115 seconds
Done87 feture(100, 1024) shape now(8800, 1024)
Time Taken to get prediction: 0.0030952670003898675 seconds
Done88 feture(100, 1024) shape now(8900, 1024)
Time Taken to get prediction: 0.00330013799975859 seconds
Done89 feture(100, 1024) shape now(9000, 1024)
Time Taken to get prediction: 0.003463752000243403 seconds
Done90 feture(100, 1024) shape now(9100, 1

Done158 feture(100, 1024) shape now(15900, 1024)
Time Taken to get prediction: 0.0035011300005862722 seconds
Done159 feture(100, 1024) shape now(16000, 1024)
Time Taken to get prediction: 0.0032073999991553137 seconds
Done160 feture(100, 1024) shape now(16100, 1024)
Time Taken to get prediction: 0.0031312339997384697 seconds
Done161 feture(100, 1024) shape now(16200, 1024)
Time Taken to get prediction: 0.0031104300014703767 seconds
Done162 feture(100, 1024) shape now(16300, 1024)
Time Taken to get prediction: 0.0031009100002847845 seconds
Done163 feture(100, 1024) shape now(16400, 1024)
Time Taken to get prediction: 0.0031633230009902036 seconds
Done164 feture(100, 1024) shape now(16500, 1024)
Time Taken to get prediction: 0.0033406889997422695 seconds
Done165 feture(100, 1024) shape now(16600, 1024)
Time Taken to get prediction: 0.0032183309995161835 seconds
Done166 feture(100, 1024) shape now(16700, 1024)
Time Taken to get prediction: 0.003357968000273104 seconds
Done167 feture(100, 

Done236 feture(100, 1024) shape now(23700, 1024)
Time Taken to get prediction: 0.005197567999857711 seconds
Done237 feture(100, 1024) shape now(23800, 1024)
Time Taken to get prediction: 0.0035053609990427503 seconds
Done238 feture(100, 1024) shape now(23900, 1024)
Time Taken to get prediction: 0.004625270999895292 seconds
Done239 feture(100, 1024) shape now(24000, 1024)
Time Taken to get prediction: 0.0033064849994843826 seconds
Done240 feture(100, 1024) shape now(24100, 1024)
Time Taken to get prediction: 0.0033928770008060383 seconds
Done241 feture(100, 1024) shape now(24200, 1024)
Time Taken to get prediction: 0.0031428709989995696 seconds
Done242 feture(100, 1024) shape now(24300, 1024)
Time Taken to get prediction: 0.003116424999461742 seconds
Done243 feture(100, 1024) shape now(24400, 1024)
Time Taken to get prediction: 0.0032091630000650184 seconds
Done244 feture(100, 1024) shape now(24500, 1024)
Time Taken to get prediction: 0.003382297998541617 seconds
Done245 feture(100, 102

Done315 feture(100, 1024) shape now(31600, 1024)
Time Taken to get prediction: 0.0031104310000955593 seconds
Done316 feture(100, 1024) shape now(31700, 1024)
Time Taken to get prediction: 0.0032828599996719277 seconds
Done317 feture(100, 1024) shape now(31800, 1024)
Time Taken to get prediction: 0.0031950580014381558 seconds
Done318 feture(100, 1024) shape now(31900, 1024)
Time Taken to get prediction: 0.0032292619998770533 seconds
Done319 feture(100, 1024) shape now(32000, 1024)
Time Taken to get prediction: 0.0033174160016642418 seconds
Done320 feture(100, 1024) shape now(32100, 1024)
Time Taken to get prediction: 0.0031841269992582966 seconds
Done321 feture(100, 1024) shape now(32200, 1024)
Time Taken to get prediction: 0.003172138000081759 seconds
Done322 feture(100, 1024) shape now(32300, 1024)
Time Taken to get prediction: 0.0030360279997694306 seconds
Done323 feture(100, 1024) shape now(32400, 1024)
Time Taken to get prediction: 0.0037444350000441773 seconds
Done324 feture(100, 

Done392 feture(100, 1024) shape now(39300, 1024)
Time Taken to get prediction: 0.003848809001283371 seconds
Done393 feture(100, 1024) shape now(39400, 1024)
Time Taken to get prediction: 0.0031569759994454216 seconds
Done394 feture(100, 1024) shape now(39500, 1024)
Time Taken to get prediction: 0.0031136039997363696 seconds
Done395 feture(100, 1024) shape now(39600, 1024)
Time Taken to get prediction: 0.003390407999177114 seconds
Done396 feture(100, 1024) shape now(39700, 1024)
Time Taken to get prediction: 0.003273338999861153 seconds
Done397 feture(100, 1024) shape now(39800, 1024)
Time Taken to get prediction: 0.003960942000048817 seconds
Done398 feture(100, 1024) shape now(39900, 1024)
Time Taken to get prediction: 0.003181305999532924 seconds
Done399 feture(100, 1024) shape now(40000, 1024)
Time Taken to get prediction: 0.004074483998920186 seconds
Done400 feture(100, 1024) shape now(40100, 1024)
Time Taken to get prediction: 0.0033392789991921745 seconds
Done401 feture(100, 1024)

In [53]:
inputto_4_valid=get_feture(inputto_3_valid[0:100],model_0)
inputto_4_valid=get_prediction(inputto_3_valid,inputto_4_valid,model_0)

Time Taken to get prediction: 0.004506440000113798 seconds
Time Taken to get prediction: 0.0033029589994839625 seconds
Done1 feture(100, 1024) shape now(200, 1024)
Time Taken to get prediction: 0.003339983999467222 seconds
Done2 feture(100, 1024) shape now(300, 1024)
Time Taken to get prediction: 0.003188710999893374 seconds
Done3 feture(100, 1024) shape now(400, 1024)
Time Taken to get prediction: 0.003148865000184742 seconds
Done4 feture(100, 1024) shape now(500, 1024)
Time Taken to get prediction: 0.0033008440004778095 seconds
Done5 feture(100, 1024) shape now(600, 1024)
Time Taken to get prediction: 0.0031513339999946766 seconds
Done6 feture(100, 1024) shape now(700, 1024)
Time Taken to get prediction: 0.0029408209993562195 seconds
Done7 feture(100, 1024) shape now(800, 1024)
Time Taken to get prediction: 0.004062495001562638 seconds
Done8 feture(100, 1024) shape now(900, 1024)
Time Taken to get prediction: 0.0031876539997028885 seconds
Done9 feture(100, 1024) shape now(1000, 1024)

In [54]:
inputto_4=np.expand_dims(inputto_4, axis=1)
inputto_4_valid=np.expand_dims(inputto_4_valid, axis=1)

# forth layer model

In [56]:
K.clear_session()
dense_model = models.Sequential()
dense_model.add(Flatten(input_shape=(1,1024)))
dense_model.add(Dense(512, activation='relu'))
dense_model.add(Dense(100, activation='softmax'))
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               51300     
Total params: 576,100
Trainable params: 576,100
Non-trainable params: 0
_________________________________________________________________


In [57]:
dense_model.compile(loss=losses.categorical_crossentropy,optimizer=optimizers.Adadelta(),metrics=['accuracy'])

In [58]:
start = timeit.default_timer() 
single_layer_3_history=dense_model.fit(inputto_4, y_train,batch_size=512,epochs=50,verbose=2,validation_data=(inputto_4_valid, y_valid), shuffle=True)
dense_model.save('single_layer_3')
dense_model.save('single_layer_3.h5')
end = timeit.default_timer()
print("Time Taken to run the model:",end - start, "seconds")

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
 - 2s - loss: 1.6302 - acc: 0.6602 - val_loss: 1.7215 - val_acc: 0.5982
Epoch 2/50
 - 2s - loss: 0.2300 - acc: 0.9468 - val_loss: 1.8262 - val_acc: 0.6104
Epoch 3/50
 - 2s - loss: 0.1199 - acc: 0.9741 - val_loss: 2.9464 - val_acc: 0.5374
Epoch 4/50
 - 2s - loss: 0.6038 - acc: 0.9329 - val_loss: 2.2199 - val_acc: 0.6328
Epoch 5/50
 - 2s - loss: 0.1620 - acc: 0.9815 - val_loss: 2.0181 - val_acc: 0.6450
Epoch 6/50
 - 2s - loss: 0.1266 - acc: 0.9805 - val_loss: 2.0806 - val_acc: 0.6526
Epoch 7/50
 - 2s - loss: 0.1504 - acc: 0.9807 - val_loss: 2.1851 - val_acc: 0.6388
Epoch 8/50
 - 2s - loss: 0.0165 - acc: 0.9974 - val_loss: 2.7296 - val_acc: 0.6040
Epoch 9/50
 - 2s - loss: 0.1742 - acc: 0.9761 - val_loss: 2.3016 - val_acc: 0.6460
Epoch 10/50
 - 2s - loss: 0.3567 - acc: 0.9543 - val_loss: 2.7832 - val_acc: 0.6282
Epoch 11/50
 - 2s - loss: 0.3314 - acc: 0.9789 - val_loss: 2.6932 - val_acc: 0.6402
Epoch 12/50
 - 2s - loss: 0.5680 - a

In [61]:
with open('single_layer_3_history', 'wb') as fo:
        pickle.dump(single_layer_3_history,fo)